# Amazon Product Review Data Analysis Project
This notebook performs full data cleaning, EDA, and visualization.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import dedent
from zipfile import ZipFile

In [4]:
# 1. Load data
df = pd.read_csv('/content/amazon.csv', dtype=str)  # read everything as str initially to inspect

In [6]:
# Save original snapshot
df.head(3).to_csv(os.path.join('/content', "original_head_snapshot.csv"), index=False)

In [7]:
# Basic info
orig_shape = df.shape
cols = list(df.columns)

In [8]:
# 2. Initial cleaning & type conversions
df = df.rename(columns=lambda c: c.strip())

In [9]:
# Strip whitespace from string columns
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipython-input-1092928125.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [10]:
# Convert numeric columns that are known
def to_float_column(s):
    if s is None:
        return None
    s = s.replace('৳', '').replace('$', '').replace(',', '').replace('%','').strip()
    try:
        return float(s)
    except:
        return np.nan

data = df.copy()

In [11]:
for col in ["discounted_price","actual_price","discount_percentage","rating","rating_count"]:
    if col in data.columns:
        if col == "rating_count":
            cleaned = data[col].str.replace(r'[^0-9]', '', regex=True)
            data[col+'_num'] = pd.to_numeric(cleaned, errors='coerce').astype('Int64')
        elif col in ["discounted_price","actual_price"]:
            cleaned = data[col].str.replace(r'[^0-9\.]', '', regex=True)
            data[col+'_num'] = pd.to_numeric(cleaned, errors='coerce')
        elif col == "discount_percentage":
            cleaned = data[col].str.replace(r'[^0-9\.]', '', regex=True)
            data[col+'_num'] = pd.to_numeric(cleaned, errors='coerce')
        elif col == "rating":
            cleaned = data[col].str.extract(r'([0-9]+\.?[0-9]*)')[0]
            data[col+'_num'] = pd.to_numeric(cleaned, errors='coerce')

In [12]:
# Standardize id columns
for idcol in ["product_id","user_id","review_id"]:
    if idcol in data.columns:
        data[idcol] = data[idcol].astype(str)

In [13]:
# Remove duplicates
before_dups = data.shape[0]
data = data.drop_duplicates()
after_dups = data.shape[0]

if "review_id" in data.columns:
    before_rdup = data.shape[0]
    data = data.drop_duplicates(subset=["review_id"])
    after_rdup = data.shape[0]

In [14]:
# Missing value summary
missing_summary = {
    col: int(data[col+'_num'].isna().sum()) if col+'_num' in data.columns else None
    for col in ["discounted_price","actual_price","discount_percentage","rating","rating_count"]
}

In [16]:
# Imputation
if "rating_num" in data.columns:
    if "category" in data.columns:
        cat_mean = data.groupby("category")['rating_num'].transform('mean')
        data['rating_num'] = data['rating_num'].fillna(cat_mean)
    overall_mean = data['rating_num'].mean()
    data['rating_num'] = data['rating_num'].fillna(overall_mean)

if "rating_count_num" in data.columns:
    if "category" in data.columns:
        cat_med = data.groupby("category")['rating_count_num'].transform('median')
        data['rating_count_num'] = data['rating_count_num'].fillna(cat_med)
    overall_med = data['rating_count_num'].median()
    data['rating_count_num'] = data['rating_count_num'].fillna(overall_med)
    data['rating_count_num'] = data['rating_count_num'].fillna(0).astype('Int64')

In [17]:
# Derived columns
if 'actual_price_num' in data.columns and 'discounted_price_num' in data.columns:
    data['discount_amount'] = data['actual_price_num'] - data['discounted_price_num']
    data['discount_rate_calc'] = np.where(data['actual_price_num']>0,
                                          data['discount_amount'] / data['actual_price_num'] * 100,
                                          np.nan)

In [18]:
# Summary stats
summary_stats = data[['discounted_price_num','actual_price_num','discount_percentage_num','rating_num','rating_count_num']].describe(include='all')

In [19]:
# Top categories
if 'category' in data.columns:
    top_categories = data['category'].value_counts().head(15)
else:
    top_categories = pd.Series()

In [20]:
# Top products by rating_count
top_products = data.sort_values('rating_count_num', ascending=False).head(15)[['product_id','product_name','category','rating_num','rating_count_num','discounted_price_num']]

In [22]:
# Save cleaned data
output_dir = '/content/'
cleaned_csv_path = os.path.join(output_dir, "cleaned_data.csv")
data.to_csv(cleaned_csv_path, index=False)

In [23]:
# Visualizations
plt.ioff()

In [24]:
# Histograms
data['discounted_price_num'].dropna().plot(kind='hist', bins=40, title='Distribution of Discounted Price')
plt.xlabel('Discounted Price')
plt.savefig(os.path.join(output_dir, "hist_discounted_price.png"))
plt.close()

data['rating_num'].dropna().plot(kind='hist', bins=20, title='Distribution of Ratings')
plt.xlabel('Rating (stars)')
plt.savefig(os.path.join(output_dir, "hist_rating.png"))
plt.close()

In [25]:
# Bar chart of top categories
if len(top_categories)>0:
    top_categories[::-1].plot(kind='barh', title='Top Categories by Count (top 15)')
    plt.xlabel('Count')
    plt.savefig(os.path.join(output_dir, "bar_top_categories.png"))
    plt.close()

In [26]:
# Scatter: rating vs rating_count
plt.scatter(data['rating_count_num'].astype(float), data['rating_num'].astype(float), alpha=0.6, s=12)
plt.xscale('log')
plt.title('Rating vs Rating Count')
plt.xlabel('Rating Count (log scale)')
plt.ylabel('Rating')
plt.savefig(os.path.join(output_dir, "scatter_rating_vs_count.png"))
plt.close()

In [27]:
# Boxplot by category (top 8 categories)
if 'discounted_price_num' in data.columns and 'category' in data.columns:
    top8 = list(top_categories.head(8).index)
    subset = data[data['category'].isin(top8)]
    subset.boxplot(column='discounted_price_num', by='category', rot=45)
    plt.title('Price Distribution by Category (top 8)')
    plt.suptitle('')
    plt.xlabel('Category')
    plt.ylabel('Discounted Price')
    plt.savefig(os.path.join(output_dir, "box_price_by_category.png"))
    plt.close()

In [28]:
# Scatter: discount_rate_calc vs rating
if 'discount_rate_calc' in data.columns:
    plt.scatter(data['discount_rate_calc'].fillna(0), data['rating_num'].astype(float), alpha=0.5, s=10)
    plt.title('Calculated Discount Rate vs Rating')
    plt.xlabel('Discount Rate (%)')
    plt.ylabel('Rating')
    plt.savefig(os.path.join(output_dir, "scatter_discount_vs_rating.png"))
    plt.close()

In [29]:
# Bar chart: top products by rating count
if top_products.shape[0]>0:
    top_products.set_index('product_id')['rating_count_num'].plot(kind='barh', title='Top Products by Rating Count (top 15)')
    plt.xlabel('Rating Count')
    plt.savefig(os.path.join(output_dir, "bar_top_products_by_rating_count.png"))
    plt.close()

print("Analysis complete. Check the output_dir for results.")

Analysis complete. Check the output_dir for results.
